In [1]:
import pyspark

In [2]:
from pyspark.sql import SparkSession

In [21]:
from pyspark.sql.functions import col, count, sum, when, countDistinct, min, max, avg, round, lit, row_number, desc, asc
from pyspark.sql.window import Window

In [4]:
# Criação do ambiente Spark
spark = SparkSession.builder \
.appName('Prata1') \
.getOrCreate()

In [5]:
df1 = spark.read.parquet(r"C:\Users\fred\Documents\CursoFullStack\Projeto\Web Scraping\Dados\df1_tratado.parquet")

In [6]:
df3 = spark.read.parquet(r"C:\Users\fred\Documents\CursoFullStack\Projeto\Web Scraping\Dados\df3_tratado.parquet")

In [8]:
df4 = spark.read.csv(r"C:\Users\fred\Documents\CursoFullStack\Projeto\Web Scraping\Dados\URLs_por_Produto.csv")

In [7]:
df1.show()

+----+--------------------+--------------------+--------------------+------+---------+----------------+-----------------+
|  ID|           Categoria|             Produto|               Marca| Preco|Avaliacao|Quant_Avaliacoes|Quant_Comentarios|
+----+--------------------+--------------------+--------------------+------+---------+----------------+-----------------+
|1709|               Camas|Colchonete Pet Ca...|            Binnopet| 179.9|      5.0|               7|                2|
|2041|               Camas|Cama Nuvem Pelúci...|             Tinenen|  36.0|      4.7|              19|                3|
|1389|               Racao|Pet Delícia Alime...|         Pet Delícia| 30.79|      4.9|             369|               49|
|1984|           Biscoitos|Snack Natural Des...|           PetTreats| 29.71|      4.1|              15|                3|
| 542|               Ossos|Osso Mordedor Com...|           Lumen Pet| 61.92|      4.7|              15|                2|
| 320|               Ban

In [8]:
tabela_categoria = df1.groupBy('Categoria').agg(
countDistinct('ID').alias('total_produtos'),
avg('Preco').alias('preco_medio'),
max('Preco').alias('mais_caro'),
min('Preco').alias('mais_barato'),
avg('Avaliacao').alias('avaliacao_media'),
avg('Quant_Avaliacoes').alias('Quant_media_aval'),
avg('Quant_Comentarios').alias('Quant_media_comen'),
max('Avaliacao').alias('melhor_aval'),
min('Avaliacao').alias('pior_aval'),)


In [9]:
tabela_categoria.show()

+--------------------+--------------+------------------+---------+-----------+------------------+------------------+------------------+-----------+---------+
|           Categoria|total_produtos|       preco_medio|mais_caro|mais_barato|   avaliacao_media|  Quant_media_aval| Quant_media_comen|melhor_aval|pior_aval|
+--------------------+--------------+------------------+---------+-----------+------------------+------------------+------------------+-----------+---------+
|                 Kit|           237|124.70415228908345|   661.04|      15.62| 4.651758813378799|   87.042194092827|27.940928270042193|        5.0|      3.0|
|               Sache|             4| 42.67000102996826|     56.0|      30.79| 4.850000023841858|            109.75|              14.5|        5.0|      4.7|
|           Repelente|             1|44.900001525878906|     44.9|       44.9| 4.300000190734863|              18.0|               2.0|        4.3|      4.3|
|              Tapete|           219|116.78779840906

In [13]:
tabela_categoria.write.parquet(r"C:\Users\fred\Documents\CursoFullStack\Projeto\Web Scraping\Dados\dados_enriquecidos\tabela_categoria")

In [9]:
totais = df1.select(count(col('ID')).alias('Produtos'),
                   count(col('Categoria')).alias('Categorias'),
                    count(col('Marca')).alias('Marca'),
                   count(col('Preco')).alias('Preco'),
                   count(col('Avaliacao')).alias('Avaliacao'),
                   count(col('Quant_Avaliacoes')).alias('Quant_Avaliacoes'),
                   count(col('Quant_Comentarios')).alias('Quant_Comentarios'))

In [15]:
totais.show()

+--------+----------+-----+-----+---------+----------------+-----------------+
|Produtos|Categorias|Marca|Preco|Avaliacao|Quant_Avaliacoes|Quant_Comentarios|
+--------+----------+-----+-----+---------+----------------+-----------------+
|    2261|      2261| 2249| 2247|     1942|            2261|             2261|
+--------+----------+-----+-----+---------+----------------+-----------------+



In [10]:
# Suponha que você já tenha esse único row
totals = totais.collect()[0].asDict()

# Converte para lista de tuplas
metricas_lista = [(k, v) for k, v in totals.items()]

# Cria o novo dataframe
df_metricas = spark.createDataFrame(metricas_lista, ["Metrica", "Total"])

# Visualiza
df_metricas.show(truncate=False)

+-----------------+-----+
|Metrica          |Total|
+-----------------+-----+
|Produtos         |2261 |
|Categorias       |2261 |
|Marca            |2249 |
|Preco            |2247 |
|Avaliacao        |1942 |
|Quant_Avaliacoes |2261 |
|Quant_Comentarios|2261 |
+-----------------+-----+



In [30]:
df_metricas.write.parquet(r"C:\Users\fred\Documents\CursoFullStack\Projeto\Web Scraping\Dados\dados_enriquecidos\totais")

In [11]:
tabela_marca = df1.groupBy('Marca').agg(count('Produto').alias('produtos'),
                                      avg('Preco').alias('preco_medio'),
                                       avg('Avaliacao').alias('avaliacao_media'),
                                       avg('Quant_Avaliacoes').alias('quant_Avaliacoes_media'),
                                       avg('Quant_Comentarios').alias('quant_Comentarios_media'),
                                        max('Preco').alias('ticket_maximo'),
                                       min('Preco').alias('ticket_minimo'),
                                        max('Avaliacao').alias('avaliacao_maxima'),
                                       min('Avaliacao').alias('avaliacao_minima'),
                                       sum('Quant_Avaliacoes').alias('quant_Avaliacoes'),
                                       sum('Quant_Comentarios').alias('quant_Comentarios'))

In [14]:
tabela_marca.columns

['Marca',
 'produtos',
 'preco_medio',
 'avaliacao_media',
 'quant_Avaliacoes_media',
 'quant_Comentarios_media',
 'ticket_maximo',
 'ticket_minimo',
 'avaliacao_maxima',
 'avaliacao_minima',
 'quant_Avaliacoes',
 'quant_Comentarios']

In [64]:
tabela_marca.write.parquet(r"C:\Users\fred\Documents\CursoFullStack\Projeto\Web Scraping\Dados\dados_enriquecidos\tabela_marca")

In [12]:
top5_avaliacao_produtos = df1.orderBy('Avaliacao','Quant_Avaliacoes', ascending=False).limit(5)

In [18]:
top5_avaliacao_produtos.show(truncate=True)

['ID',
 'Categoria',
 'Produto',
 'Marca',
 'Preco',
 'Avaliacao',
 'Quant_Avaliacoes',
 'Quant_Comentarios']

In [13]:
bot5_avaliacao_produtos = df1.filter(col('Avaliacao').isNotNull()).orderBy('Avaliacao', ascending=True).limit(5)

In [21]:
bot5_avaliacao_produtos.columns

['ID',
 'Categoria',
 'Produto',
 'Marca',
 'Preco',
 'Avaliacao',
 'Quant_Avaliacoes',
 'Quant_Comentarios']

In [14]:
top5_quant_avaliacao_produtos = df1.orderBy('Quant_Avaliacoes', ascending=False).limit(5)
top5_quant_comentarios_produtos = df1.orderBy('Quant_Comentarios', ascending=False).limit(5)
bot5_quant_avaliacao_produtos = df1.filter((col('Quant_Avaliacoes') > 0)).orderBy('Quant_Avaliacoes', ascending=True).limit(5)
bot5_quant_comentarios_produtos = df1.filter((col('Quant_Comentarios') > 0)).orderBy('Quant_Comentarios', ascending=True).limit(5)

In [24]:
top5_quant_avaliacao_produtos.show()
top5_quant_comentarios_produtos.show()
bot5_quant_avaliacao_produtos.show()
bot5_quant_comentarios_produtos.show()

+----+---------+--------------------+-------+------+---------+----------------+-----------------+
|  ID|Categoria|             Produto|  Marca| Preco|Avaliacao|Quant_Avaliacoes|Quant_Comentarios|
+----+---------+--------------------+-------+------+---------+----------------+-----------------+
|1408|   Tapete|X2 Tapete Higiêni...|Petlike| 119.0|      4.4|            6715|                1|
| 525|   Tapete|Tapete higiênico ...|Petlike|  56.0|      4.4|            6712|                1|
|2211|   Tapete|Petlike Good Pad ...|Petlike|  79.0|      4.4|            6711|                1|
|  17|    Racao|Alimento Golden P...| Golden|179.99|      4.9|            6192|                1|
|1337|    Racao|PremieRpet Golden...| Golden|  26.0|      4.9|            6185|                1|
+----+---------+--------------------+-------+------+---------+----------------+-----------------+

+----+---------+--------------------+---------------+------+---------+----------------+-----------------+
|  ID|Categ

In [15]:
marca_mais_avaliada = tabela_marca.orderBy('quant_Avaliacoes', ascending=False).select(col('Marca').alias('mais_avaliada')).limit(1)
marca_menos_avaliada = tabela_marca.orderBy('quant_Avaliacoes', ascending=True).select(col('Marca').alias('menos_avaliada')).limit(1)
marca_menos_comentarios = tabela_marca.orderBy('quant_Comentarios', ascending=True).select(col('Marca').alias('menos_comentarios')).limit(1)
marca_mais_comentarios = tabela_marca.orderBy('quant_Comentarios', ascending=False).select(col('Marca').alias('mais_comentarios')).limit(1)

In [27]:
marca_mais_avaliada.show()
marca_menos_avaliada.show()
marca_menos_comentarios.show()
marca_mais_comentarios.show()

+-------------+
|mais_avaliada|
+-------------+
|       Golden|
+-------------+

+---------------+
| menos_avaliada|
+---------------+
|Bezzie Pet Food|
+---------------+

+-----------------+
|menos_comentarios|
+-----------------+
|      Maxximo Pet|
+-----------------+

+----------------+
|mais_comentarios|
+----------------+
|     Special Dog|
+----------------+



In [16]:
tabela_extremos = marca_mais_avaliada.join(marca_menos_avaliada).join(marca_menos_comentarios).join(marca_mais_comentarios)

In [30]:
tabela_extremos.show()

+-------------+---------------+-----------------+----------------+
|mais_avaliada| menos_avaliada|menos_comentarios|mais_comentarios|
+-------------+---------------+-----------------+----------------+
|       Golden|Bezzie Pet Food|      Maxximo Pet|     Special Dog|
+-------------+---------------+-----------------+----------------+



In [40]:
df1.columns

['ID',
 'Categoria',
 'Produto',
 'Marca',
 'Preco',
 'Avaliacao',
 'Quant_Avaliacoes',
 'Quant_Comentarios']

In [11]:
df_fato_produto = df1 \
    .withColumn("volume_total_interacoes", col("Quant_Avaliacoes") + col("Quant_Comentarios")) \
    .withColumn("avaliacao_por_preco", round(col("Avaliacao") / col("Preco"), 2)) \
    .withColumn("comentarios_por_avaliacoes", when(col("Quant_Avaliacoes") > 0, round(col("Quant_Comentarios") / col("Quant_Avaliacoes"), 2)).otherwise(0)) \
    .withColumn("flag_pouco_comentado",when(col("Quant_Comentarios") < 5, "Sim").otherwise("Não")) \
    .withColumn("flag_pouco_avaliado",when(col("Quant_Avaliacoes") < 5, "Sim").otherwise("Não")) 

In [12]:
df_fato_produto = df_fato_produto.withColumn(
    "flag_potencial_oculto",
    when((col("Avaliacao") >= 4.5) & (col("volume_total_interacoes") < 10), "Sim").otherwise("Não")
).withColumn(
    "flag_risco_imagem",
    when((col("Avaliacao") <= 3.0) & (col("Quant_comentarios") > 10), "Sim").otherwise("Não")
)


In [32]:
df_fato_produto.write.parquet(r"C:\Users\fred\Documents\CursoFullStack\Projeto\Web Scraping\Dados\dados_enriquecidos\df_fato_produto")

In [35]:
df_dim_categoria = df_fato_produto.groupBy("Categoria").agg(
    count("ID").alias("quantidade_produtos"),
    avg("Avaliacao").alias("avaliacao_media_categoria"),
    sum("Quant_Avaliacoes").alias("quantidade_total_avaliacoes"),
    sum("Quant_Comentarios").alias("quantidade_total_comentarios"),
    avg("Preco").alias("preco_medio_categoria")
)


In [36]:
df_dim_categoria.write.parquet(r"C:\Users\fred\Documents\CursoFullStack\Projeto\Web Scraping\Dados\dados_enriquecidos\df_dim_categoria")

In [46]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.functions import col, desc, asc, row_number, lit, avg

# 1. Adicionar coluna score_confianca
C = df_fato_produto.select(avg("Avaliacao")).first()[0]  # média global
m = 25  # mínimo de avaliações para confiança

df_fato_produto = df_fato_produto.withColumn(
    "score_confianca",
    (col("Quant_Avaliacoes") / (col("Quant_Avaliacoes") + lit(m))) * col("Avaliacao") +
    (lit(m) / (col("Quant_Avaliacoes") + lit(m))) * lit(C)
)

# 2. Função para gerar extremos por categoria
def gerar_extremos(df, coluna_alvo, tipo_ordem, top_n=3, desempate_por=None):
    if tipo_ordem == "top":
        ordem = [desc(coluna_alvo)]
        if desempate_por:
            ordem.append(desc(desempate_por))
    else:
        ordem = [asc(coluna_alvo)]
        if desempate_por:
            ordem.append(asc(desempate_por))
    
    janela = Window.partitionBy("Categoria").orderBy(*ordem)

    return df.withColumn("rank", row_number().over(janela)) \
             .filter(col("rank") <= top_n) \
             .withColumn("tipo_extremo", lit(f"{tipo_ordem}_{coluna_alvo}")) \
             .drop("rank")

# 3. Gerar a tabela final com todos os extremos
df_dim_extremos = gerar_extremos(df_fato_produto, "score_confianca", "top", desempate_por="Quant_Avaliacoes") \
    .union(gerar_extremos(df_fato_produto, "score_confianca", "bot", desempate_por="Quant_Avaliacoes")) \
    .union(gerar_extremos(df_fato_produto, "Preco", "top", desempate_por="Avaliacao")) \
    .union(gerar_extremos(df_fato_produto, "Preco", "bot", desempate_por="Avaliacao")) \
    .union(gerar_extremos(df_fato_produto, "Quant_Avaliacoes", "top", desempate_por="Avaliacao")) \
    .union(gerar_extremos(df_fato_produto, "Quant_Avaliacoes", "bot", desempate_por="Avaliacao")) \
    .union(gerar_extremos(df_fato_produto, "Quant_Comentarios", "top", desempate_por="Quant_Avaliacoes")) \
    .union(gerar_extremos(df_fato_produto, "Quant_Comentarios", "bot", desempate_por="Quant_Avaliacoes"))


In [47]:
# Salvar o DataFrame consolidado em um único arquivo CSV
df_dim_extremos.coalesce(1).write.parquet(r"C:\Users\fred\Documents\CursoFullStack\Projeto\Web Scraping\Dados\dados_enriquecidos\df_dim_extremos")

In [43]:
df_dim_marca = df_fato_produto.groupBy("Marca").agg(
    countDistinct("Categoria").alias("quant_categorias"),  # Em quantas categorias essa marca atua
    count("Produto").alias("quant_produtos_marca"),        # Quantos produtos ela tem no total
    sum("Quant_Avaliacoes").alias("total_avaliacoes_marca"),  # Soma das avaliações
    sum("Quant_Comentarios").alias("total_comentarios_marca"), # Soma dos comentários
    max("Preco").alias("preco_max_marca"),
    min("Preco").alias("preco_min_marca")
)


In [44]:
df_dim_marca.write.parquet(r"C:\Users\fred\Documents\CursoFullStack\Projeto\Web Scraping\Dados\dados_enriquecidos\df_dim_marca")

In [48]:
from pyspark.sql.functions import when, concat_ws, lit

# Adicionar flag de presença em extremos
extremos_ids = df_dim_extremos.select("ID").distinct()
df_com_extremos = df_fato_produto.join(extremos_ids, on="ID", how="left") \
                                 .withColumn("Posicionado_em", when(F.col("ID").isNotNull(), "Sim").otherwise("Não"))

# Adicionar flag para análise manual baseada em risco ou potencial oculto
df_dim_produto = df_com_extremos.withColumn(
    "Flag_analise_manual",
    when((F.col("flag_risco_imagem") == "Sim") | (F.col("flag_potencial_oculto") == "Sim"), "Sim").otherwise("Não")
).select(
    "ID", "Produto", "Marca", "Categoria", "Preco", "Avaliacao",
    "Quant_Avaliacoes", "Quant_Comentarios", "Posicionado_em", "Flag_analise_manual"
)


In [49]:
df_dim_produto.write.parquet(r"C:\Users\fred\Documents\CursoFullStack\Projeto\Web Scraping\Dados\dados_enriquecidos\df_dim_produto")

In [ ]:
df_dim_comportamento = 